<a href="https://colab.research.google.com/github/Cinichi/Ani-Downloader/blob/main/anime_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# ================================================================
# 📜 DISCLAIMER
# Personal / educational use only.
# Respect copyright laws and the AnimeKai/host site terms of use.
# Do NOT use this notebook for commercial or infringing purposes.
# ================================================================

# 🎬 AnimeKai Episode Downloader & Merger (FIXED VERSION)
# Fixes: Subtitle embedding for Soft Sub/Dub, Single episode upload, Episode naming

# @title 🔧 Install Dependencies { display-mode: "form" }
print("📦 Installing required packages...")
!pip install -q requests beautifulsoup4 cloudscraper m3u8 pycryptodome tqdm yt-dlp
!apt-get -qq install -y ffmpeg aria2 > /dev/null 2>&1
print("✅ All dependencies installed!\n")

# @title ⚙️ Configuration { display-mode: "form" }

#@markdown ### 🔗 Anime URL
anime_url = "https://anikai.to/watch/my-dress-up-darling-season-2-p8em"  # @param {type:"string"}

#@markdown ### 📺 Episode Selection
download_mode = "All Episodes"  # @param ["All Episodes", "Episode Range", "Single Episode"]

#@markdown Single episode (accepts things like "12.5", "SP"):
single_episode = "1"  # @param {type:"string"}

#@markdown Episode range (interpreted numerically when possible):
start_episode = "1"  # @param {type:"string"}
end_episode   = "2"  # @param {type:"string"}

#@markdown ### 🎥 Quality & Audio Settings
video_quality = "720p"  # @param ["1080p", "720p", "480p", "360p"]

# NOTE: "Dub (with subs)" is AnimeKai's dub type; not true dual-audio.
prefer_type = "Soft Sub"  # @param ["Hard Sub", "Soft Sub", "Dub (with subs)"]
prefer_server = "Server 1"  # @param ["Server 1", "Server 2"]

#@markdown ### 📥 Download Settings
download_method = "yt-dlp"  # @param ["yt-dlp", "aria2", "chunks", "ffmpeg"]

chunk_size_mb      = 15    # @param {type:"slider", min:1, max:20, step:1}
max_workers        = 15    # @param {type:"slider", min:1, max:16, step:1}
max_retries        = 7    # @param {type:"slider", min:1, max:10, step:1}
connection_timeout = 300  # @param {type:"slider", min:60, max:600, step:30}

#@markdown ### 🔗 Merge Settings
merge_episodes = True  # @param {type:"boolean"}
season_number  = 0     # @param {type:"integer"}  # 0 = auto-detect
keep_individual_files = False  # @param {type:"boolean"}

#@markdown ### 📤 Upload Settings
upload_destination = "Both"  # @param ["GoFile.io Only", "Google Drive Only", "Both", "None (Keep Local)"]
upload_merged_only = True    # @param {type:"boolean"}

print("✅ Configuration set!")
print(f"📥 Download method: {download_method}")
print(f"⚙️ Workers: {max_workers} | Chunk size: {chunk_size_mb}MB")
print(f"🔄 Max retries: {max_retries} | Timeout: {connection_timeout}s")
if merge_episodes:
    print(f"🔗 Merge enabled | Keep files: {keep_individual_files}")

# ================================================================
# Imports & globals
# ================================================================

import requests
import re
import json
import os
import time
import subprocess
from typing import List, Optional, Tuple, Dict, Any

from bs4 import BeautifulSoup
import cloudscraper
from urllib.parse import urlparse
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def log(level: str, msg: str) -> None:
    print(f"[{level}] {msg}")

BASE_URL = "https://animekai.to"
scraper = cloudscraper.create_scraper(
    browser={"browser": "chrome", "platform": "windows", "desktop": True}
)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Referer": BASE_URL,
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
}

RETRY_CONFIG = {
    "max_retries": max_retries,
    "sleep_between": 3,
    "timeout": connection_timeout,
}

# ================================================================
# enc-dec helpers
# ================================================================

def call_enc_dec_api(endpoint: str, payload: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    base = "https://enc-dec.app/api"
    url = f"{base}/{endpoint}"
    try:
        if endpoint.startswith("enc-"):
            text = payload.get("text", "")
            resp = scraper.get(f"{url}?text={text}", headers=HEADERS, timeout=15)
        else:
            resp = scraper.post(
                url,
                headers={"Content-Type": "application/json"},
                data=json.dumps(payload),
                timeout=30,
            )
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        log("ERROR", f"enc-dec API '{endpoint}' failed: {e}")
        return None

def enc_kai(text: str) -> Optional[str]:
    data = call_enc_dec_api("enc-kai", {"text": text})
    if not data or "result" not in data:
        log("ERROR", "Failed to get enc-kai result.")
        return None
    return data["result"]

def dec_kai(text: str) -> Optional[Dict[str, Any]]:
    data = call_enc_dec_api("dec-kai", {"text": text})
    if not data or "result" not in data:
        log("ERROR", "Failed to decode dec-kai payload.")
        return None
    return data["result"]

def dec_mega(text: str, agent: str) -> Optional[Dict[str, Any]]:
    data = call_enc_dec_api("dec-mega", {"text": text, "agent": agent})
    if not data or "result" not in data:
        log("ERROR", "Failed to decode dec-mega payload.")
        return None
    return data["result"]

# ================================================================
# Anime info & episodes
# ================================================================

def get_anime_details(url: str) -> Tuple[Optional[str], str]:
    try:
        r = scraper.get(url, headers=HEADERS, timeout=30)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        anime_div = soup.select_one("div[data-id]")
        anime_id = anime_div.get("data-id") if anime_div else None

        title_elem = (
            soup.select_one("div.title-wrapper h1.title span")
            or soup.select_one("h1.title")
            or soup.select_one(".anime-title")
        )
        title = title_elem.get("title") if title_elem and title_elem.get("title") else (
            title_elem.text.strip() if title_elem else "Unknown"
        )
        title = re.sub(r'[<>:"/\\|?*]', "", title)
        return anime_id, title
    except Exception as e:
        log("ERROR", f"Error getting anime details: {e}")
        return None, "Unknown"

def detect_season_from_title(title: str) -> int:
    patterns = [
        r"[Ss]eason\s+(\d+)",
        r"[Ss](\d+)",
        r"(\d+)(?:st|nd|rd|th)\s+[Ss]eason",
        r"\s+(\d+)$",
        r"Part\s+(\d+)",
        r"Cour\s+(\d+)",
    ]
    for p in patterns:
        m = re.search(p, title)
        if m:
            return int(m.group(1))
    return 1

def safe_episode_key(ep_id: str) -> Tuple[int, float]:
    m = re.match(r"(\d+)(?:\.(\d+))?", ep_id)
    if m:
        main = int(m.group(1))
        frac = float(f"0.{m.group(2)}") if m.group(2) else 0.0
        return main, frac
    return (10**9, 0.0)

def get_episode_list(anime_id: str) -> List[Dict[str, Any]]:
    try:
        enc = enc_kai(anime_id)
        if not enc:
            return []
        url = f"{BASE_URL}/ajax/episodes/list?ani_id={anime_id}&_={enc}"
        r = scraper.get(url, headers=HEADERS, timeout=30)
        r.raise_for_status()
        data = r.json()
        html = data.get("result", "")
        if not html:
            return []

        soup = BeautifulSoup(html, "html.parser")
        episodes: List[Dict[str, Any]] = []
        for ep in soup.select("div.eplist a"):
            token = ep.get("token", "")
            ep_id = ep.get("num", "").strip()
            langs = ep.get("langs", "0")
            try:
                langs_int = int(langs)
            except ValueError:
                langs_int = 0
            if langs_int == 1:
                subdub = "Sub"
            elif langs_int == 3:
                subdub = "Dub & Sub"
            else:
                subdub = ""

            episodes.append(
                {
                    "id": ep_id,
                    "sort_key": safe_episode_key(ep_id),
                    "token": token,
                    "subdub": subdub,
                    "title": f"Episode {ep_id}",
                }
            )
        episodes.sort(key=lambda e: e["sort_key"])
        return episodes
    except Exception as e:
        log("ERROR", f"Error getting episodes: {e}")
        return []

# ================================================================
# Server selection
# ================================================================

def normalize(s: str) -> str:
    return s.strip().lower()

def server_matches_pref(server_name: str, preferred: str) -> bool:
    s = normalize(server_name)
    p = normalize(preferred)
    return p in s or s in p

def get_video_servers(token: str) -> List[Dict[str, str]]:
    try:
        enc = enc_kai(token)
        if not enc:
            return []
        url = f"{BASE_URL}/ajax/links/list?token={token}&_={enc}"
        r = scraper.get(url, headers=HEADERS, timeout=30)
        r.raise_for_status()
        data = r.json()
        html = data.get("result", "")
        if not html:
            return []
        soup = BeautifulSoup(html, "html.parser")
        servers: List[Dict[str, str]] = []
        for type_div in soup.select("div.server-items[data-id]"):
            type_id = type_div.get("data-id", "")
            for server in type_div.select("span.server[data-lid]"):
                server_id = server.get("data-lid", "")
                server_name = server.text.strip()
                servers.append(
                    {"type": type_id, "server_id": server_id, "server_name": server_name}
                )
        return servers
    except Exception as e:
        log("ERROR", f"Error getting servers: {e}")
        return []

def choose_server(
    servers: List[Dict[str, str]],
    prefer_type_label: str,
    prefer_server_name: str,
) -> Optional[Dict[str, str]]:
    type_map = {
        "Hard Sub":        "sub",
        "Soft Sub":        "softsub",
        "Dub (with subs)": "dub",
    }
    prefer_type_id = type_map.get(prefer_type_label, "softsub")

    if not servers:
        return None

    cand = [
        s for s in servers
        if s["type"] == prefer_type_id and server_matches_pref(s["server_name"], prefer_server_name)
    ]
    if cand:
        return cand[0]

    cand = [s for s in servers if server_matches_pref(s["server_name"], prefer_server_name)]
    if cand:
        return cand[0]

    cand = [s for s in servers if s["type"] == prefer_type_id]
    if cand:
        return cand[0]

    return servers[0]

# ================================================================
# Resolve video URL with subtitle tracks (FIXED)
# ================================================================

def get_video_data(server_id: str) -> Optional[Dict[str, Any]]:
    """
    Returns dict with:
      - video_url: main video URL
      - subtitles: list of subtitle tracks [{'url': ..., 'lang': ...}]
    """
    try:
        enc = enc_kai(server_id)
        if not enc:
            return None
        url = f"{BASE_URL}/ajax/links/view?id={server_id}&_={enc}"
        r = scraper.get(url, headers=HEADERS, timeout=30)
        r.raise_for_status()
        data = r.json()
        encoded_link = data.get("result", "")
        if not encoded_link:
            return None

        dec = dec_kai(encoded_link)
        if not dec:
            return None
        iframe_url = dec.get("url", "")
        if not iframe_url:
            return None

        parsed = urlparse(iframe_url)
        token = parsed.path.split("/")[-1]
        media_url = f"{parsed.scheme}://{parsed.netloc}/media/{token}"
        r2 = scraper.get(media_url, headers=HEADERS, timeout=30)
        r2.raise_for_status()
        j2 = r2.json()
        mega_token = j2.get("result", "")
        if not mega_token:
            return None

        mega = dec_mega(mega_token, HEADERS["User-Agent"])
        if not mega:
            return None

        sources = mega.get("sources", [])
        if not sources:
            return None

        video_url = sources[0].get("file", "")

        # Extract subtitle tracks (VTT only)
        subtitle_tracks = []
        tracks = mega.get("tracks", [])
        for track in tracks:
            if track.get("kind") == "captions" and track.get("file", "").endswith(".vtt"):
                subtitle_tracks.append({
                    "url": track["file"],
                    "lang": track.get("label", "Unknown")
                })

        return {
            "video_url": video_url,
            "subtitles": subtitle_tracks
        }
    except Exception as e:
        log("ERROR", f"Error getting video data: {e}")
        return None

# ================================================================
# Download with subtitle embedding (FIXED)
# ================================================================

def download_with_ytdlp(url: str, output_file: str, episode_label: str, subtitles: List[Dict] = None) -> bool:
    """Download with yt-dlp, embedding subtitles if available."""
    try:
        log("INFO", f"Using yt-dlp -> {os.path.basename(output_file)}")

        # Base command
        cmd = [
            "yt-dlp",
            url,
            "-o",
            output_file,
            "--no-warnings",
            "--no-check-certificate",
            "--concurrent-fragments",
            str(max_workers),
            "--retries",
            str(RETRY_CONFIG["max_retries"]),
            "--fragment-retries",
            str(RETRY_CONFIG["max_retries"]),
            "--socket-timeout",
            str(RETRY_CONFIG["timeout"]),
            "--user-agent",
            HEADERS["User-Agent"],
            "--referer",
            BASE_URL,
            "--newline",
        ]

        # Add subtitle handling if available
        if subtitles:
            log("INFO", f"Found {len(subtitles)} subtitle track(s)")
            # Download video and subs separately, then merge
            temp_video = output_file.replace(".mp4", "_temp.mp4")
            cmd_copy = cmd.copy()
            cmd_copy[cmd_copy.index("-o") + 1] = temp_video

            # Download video first
            proc = subprocess.Popen(
                cmd_copy,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1,
            )
            with tqdm(
                total=100,
                unit="%",
                desc=f"Ep {episode_label} (video)",
                bar_format="{desc}: {percentage:3.0f}%|{bar}| {elapsed}",
            ) as pbar:
                last = 0.0
                for line in proc.stdout:
                    m = re.search(r"\[download\]\s+(\d+\.?\d*)%", line)
                    if m:
                        cur = float(m.group(1))
                        delta = cur - last
                        if delta > 0:
                            pbar.update(delta)
                            last = cur
            proc.wait()

            if proc.returncode != 0 or not os.path.exists(temp_video):
                log("ERROR", f"Video download failed")
                return False

            # Download subtitles
            sub_files = []
            for idx, sub in enumerate(subtitles):
                sub_path = output_file.replace(".mp4", f"_sub{idx}.vtt")
                try:
                    log("INFO", f"Downloading subtitle: {sub['lang']}")
                    r = scraper.get(sub['url'], headers=HEADERS, timeout=30)
                    r.raise_for_status()
                    with open(sub_path, 'wb') as f:
                        f.write(r.content)
                    sub_files.append((sub_path, sub['lang']))
                except Exception as e:
                    log("WARN", f"Failed to download subtitle {sub['lang']}: {e}")

            # Merge video + subtitles with ffmpeg
            if sub_files:
                log("INFO", f"Embedding {len(sub_files)} subtitle(s)...")
                ffmpeg_cmd = ["ffmpeg", "-i", temp_video]

                # Add subtitle inputs
                for sub_file, _ in sub_files:
                    ffmpeg_cmd.extend(["-i", sub_file])

                # Map video and audio
                ffmpeg_cmd.extend(["-map", "0:v", "-map", "0:a"])

                # Map and set metadata for each subtitle
                for idx, (_, lang) in enumerate(sub_files, 1):
                    ffmpeg_cmd.extend([
                        "-map", f"{idx}:0",
                        f"-metadata:s:s:{idx-1}", f"language={lang[:3].lower()}",
                        f"-metadata:s:s:{idx-1}", f"title={lang}"
                    ])

                # Output settings
                ffmpeg_cmd.extend([
                    "-c:v", "copy",
                    "-c:a", "copy",
                    "-c:s", "mov_text",  # MP4-compatible subtitle codec
                    "-y",
                    output_file
                ])

                result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

                # Cleanup
                try:
                    os.remove(temp_video)
                    for sub_file, _ in sub_files:
                        os.remove(sub_file)
                except Exception:
                    pass

                if result.returncode == 0 and os.path.exists(output_file):
                    log("INFO", f"✅ Download complete with subtitles: {os.path.basename(output_file)}")
                    return True
                else:
                    log("WARN", "Subtitle embedding failed, keeping video only")
                    if os.path.exists(temp_video):
                        shutil.move(temp_video, output_file)
                    return os.path.exists(output_file)
            else:
                # No subtitles downloaded, just rename temp file
                shutil.move(temp_video, output_file)
                return True

        else:
            # No subtitles, standard download
            proc = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1,
            )
            with tqdm(
                total=100,
                unit="%",
                desc=f"Ep {episode_label}",
                bar_format="{desc}: {percentage:3.0f}%|{bar}| {elapsed}",
            ) as pbar:
                last = 0.0
                for line in proc.stdout:
                    m = re.search(r"\[download\]\s+(\d+\.?\d*)%", line)
                    if m:
                        cur = float(m.group(1))
                        delta = cur - last
                        if delta > 0:
                            pbar.update(delta)
                            last = cur
            proc.wait()
            if proc.returncode == 0 and os.path.exists(output_file):
                log("INFO", f"✅ Download complete: {os.path.basename(output_file)}")
                return True
            log("ERROR", f"yt-dlp failed with code {proc.returncode}")
            return False

    except Exception as e:
        log("ERROR", f"yt-dlp error: {e}")
        return False

def download_direct(url: str, output_file: str, episode_label: str) -> bool:
    log("INFO", f"Using direct HTTP -> {os.path.basename(output_file)}")
    try:
        with scraper.get(url, headers=HEADERS, stream=True, timeout=RETRY_CONFIG["timeout"]) as r:
            r.raise_for_status()
            total = int(r.headers.get("Content-Length", 0))
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, "wb") as f:
                if total > 0:
                    with tqdm(total=total, unit="B", unit_scale=True, desc=f"Ep {episode_label}") as pbar:
                        for chunk in r.iter_content(chunk_size=8192):
                            if not chunk:
                                continue
                            f.write(chunk)
                            pbar.update(len(chunk))
                else:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
        log("INFO", f"✅ Download complete: {os.path.basename(output_file)}")
        return True
    except Exception as e:
        log("ERROR", f"Direct download error: {e}")
        return False

def is_m3u8_url(url: str) -> bool:
    if ".m3u8" in url.split("?")[0]:
        return True
    try:
        r = scraper.head(url, headers=HEADERS, timeout=10)
        ctype = r.headers.get("Content-Type", "")
        return "application/vnd.apple.mpegurl" in ctype or "application/x-mpegURL" in ctype
    except Exception:
        return False

def download_episode(video_data: Dict[str, Any], output_file: str, episode_label: str) -> bool:
    """Download episode with subtitle support."""
    url = video_data["video_url"]
    subtitles = video_data.get("subtitles", [])

    use_m3u8 = is_m3u8_url(url)
    if download_method == "yt-dlp":
        primary = lambda u, o: download_with_ytdlp(u, o, episode_label, subtitles)
        fallback = lambda u, o: download_direct(u, o, episode_label)
    else:
        primary = lambda u, o: download_direct(u, o, episode_label)
        fallback = lambda u, o: download_direct(u, o, episode_label)

    for attempt in range(1, RETRY_CONFIG["max_retries"] + 1):
        if os.path.exists(output_file):
            os.remove(output_file)
        if attempt > 1:
            log("INFO", f"Retry {attempt}/{RETRY_CONFIG['max_retries']} for {os.path.basename(output_file)}")
        if primary(url, output_file):
            return True
        time.sleep(RETRY_CONFIG["sleep_between"])
        if attempt == RETRY_CONFIG["max_retries"] - 1:
            log("WARN", "Primary method failing; trying fallback.")
            if fallback(url, output_file):
                return True
    return False

# ================================================================
# Merge
# ================================================================

def merge_videos(
    file_list: List[str],
    anime_title: str,
    season_num: int,
    first_ep_id: str,
    last_ep_id: str,
) -> Optional[str]:
    if not file_list:
        log("ERROR", "No files to merge.")
        return None

    valid_files = [f for f in file_list if os.path.exists(f)]
    if len(valid_files) != len(file_list):
        log("ERROR", "Some input files for merging are missing.")
        return None

    merged_filename = f"{anime_title} Season {season_num:02d} Episodes {first_ep_id}-{last_ep_id}.mp4"
    merged_filename = re.sub(r'[<>:"/\\|?*]', "", merged_filename)
    merged_path = os.path.join(os.path.dirname(file_list[0]), merged_filename)

    log("INFO", f"Merging {len(valid_files)} files into {merged_filename}")

    list_file = os.path.join(os.path.dirname(file_list[0]), "filelist_merge.txt")
    try:
        with open(list_file, "w", encoding="utf-8") as f:
            for vf in valid_files:
                f.write(f"file '{os.path.abspath(vf)}'\n")

        cmd = [
            "ffmpeg",
            "-f", "concat",
            "-safe", "0",
            "-i", list_file,
            "-c:v", "copy",
            "-c:a", "copy",
            "-c:s", "copy",  # Also copy subtitle streams
            "-y",
            "-loglevel", "info",
            merged_path,
        ]
        proc = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )

        print("Merging...", end="", flush=True)
        ffmpeg_err = []
        for line in proc.stderr:
            ffmpeg_err.append(line.rstrip())
            if "time=" in line:
                print(".", end="", flush=True)
        proc.wait()
        print()

        if proc.returncode != 0 or not os.path.exists(merged_path):
            log("ERROR", "ffmpeg merge failed. Last lines:")
            for l in ffmpeg_err[-10:]:
                print(l)
            return None

        log("INFO", f"✅ Merged: {merged_filename}")
        return merged_path
    except Exception as e:
        log("ERROR", f"Merge error: {e}")
        return None
    finally:
        try:
            if os.path.exists(list_file):
                os.remove(list_file)
        except Exception:
            pass

# ================================================================
# Upload helpers
# ================================================================

def upload_to_gofile(filepath: str) -> Optional[str]:
    try:
        filename = os.path.basename(filepath)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        log("INFO", f"Uploading to GoFile: {filename} ({size_mb:.2f} MB)")

        server = None
        for attempt in range(1, 6):
            try:
                r = requests.get("https://api.gofile.io/servers", timeout=15)
                data = r.json()

                if data['status'] == 'ok' and data['data']['servers']:
                    server = data['data']['servers'][0]['name']
                    break
                else:
                    log("WARN", f"GoFile busy (Attempt {attempt}/5)... Waiting 3s.")
            except Exception:
                pass
            time.sleep(3)

        if not server:
            log("ERROR", "❌ GoFile upload failed: No servers available currently.")
            return None

        upload_url = f"https://{server}.gofile.io/contents/uploadfile"
        with open(filepath, "rb") as f:
            with tqdm(total=size_mb, unit="MB", desc="GoFile Upload") as pbar:

                class ProgressFile:
                    def __init__(self, file_obj, pbar):
                        self.f = file_obj
                        self.pbar = pbar

                    def read(self, size=-1):
                        data = self.f.read(size)
                        if not data: return data
                        self.pbar.update(len(data) / (1024 * 1024))
                        return data

                    def __getattr__(self, name):
                        return getattr(self.f, name)

                pf = ProgressFile(f, pbar)
                resp = requests.post(upload_url, files={"file": (filename, pf)}, timeout=7200)

        j = resp.json()
        if j.get("status") == "ok":
            link = j["data"]["downloadPage"]
            log("INFO", f"✅ GoFile link: {link}")
            return link
        else:
            log("ERROR", f"GoFile upload failed: {j}")
            return None

    except Exception as e:
        log("ERROR", f"GoFile upload error: {e}")
        return None

def upload_to_gdrive(filepath: str) -> Optional[str]:
    try:
        from google.colab import drive

        if not os.path.exists("/content/drive/MyDrive"):
            log("INFO", "Mounting Google Drive...")
            drive.mount("/content/drive")

        dest_dir = "/content/drive/MyDrive/AnimeKai_Downloads/"
        os.makedirs(dest_dir, exist_ok=True)
        filename = os.path.basename(filepath)
        dest_path = os.path.join(dest_dir, filename)

        total = os.path.getsize(filepath)
        log("INFO", f"Copying to GDrive: {filename}")
        with open(filepath, "rb") as src, open(dest_path, "wb") as dst:
            with tqdm(total=total, unit="B", unit_scale=True, desc="Uploading to GDrive") as pbar:
                while True:
                    chunk = src.read(8192)
                    if not chunk:
                        break
                    dst.write(chunk)
                    pbar.update(len(chunk))

        log("INFO", f"✅ Uploaded to {dest_path}")
        return dest_path
    except Exception as e:
        log("ERROR", f"GDrive upload error: {e}")
        return None

# ================================================================
# Filename generator (NEW - FIXED)
# ================================================================

def generate_episode_filename(anime_title: str, season_num: int, ep_id: str) -> str:
    """
    Generate proper episode filename: Anime Name Season xx Episode xx.mp4
    Example: My Dress Up Darling Season 02 Episode 01.mp4
    """
    # Pad episode number if it's a simple integer
    try:
        ep_num = float(ep_id)
        if ep_num == int(ep_num):
            # Simple integer episode (1, 2, 3...)
            ep_formatted = f"{int(ep_num):02d}"
        else:
            # Decimal episode (12.5, etc.)
            ep_formatted = ep_id
    except ValueError:
        # Special episodes (SP, OVA, etc.)
        ep_formatted = ep_id

    filename = f"{anime_title} Season {season_num:02d} Episode {ep_formatted}.mp4"
    # Remove invalid characters
    filename = re.sub(r'[<>:"/\\|?*]', "", filename)
    return filename

# ================================================================
# main() - FIXED for proper episode naming
# ================================================================

def parse_episode_id(ep_id: str) -> Tuple[int, float]:
    return safe_episode_key(ep_id)

def in_episode_range(ep_id: str, start_id: str, end_id: str) -> bool:
    s_key = parse_episode_id(start_id)
    e_key = parse_episode_id(end_id)
    k = parse_episode_id(ep_id)
    return s_key <= k <= e_key

def main():
    print("\n" + "=" * 70)
    print("🎬 ANIMEKAI EPISODE DOWNLOADER & MERGER (FIXED)")
    print("=" * 70)

    log("INFO", f"Processing: {anime_url}")

    anime_id, anime_title = get_anime_details(anime_url)
    if not anime_id:
        raise RuntimeError("Could not extract anime ID – check URL or site changes.")

    log("INFO", f"Anime ID: {anime_id}")
    log("INFO", f"Title: {anime_title}")

    detected_season = detect_season_from_title(anime_title)
    final_season = season_number if season_number > 0 else detected_season
    log("INFO", f"Season: {final_season} ({'auto' if season_number == 0 else 'manual'})")

    episodes = get_episode_list(anime_id)
    if not episodes:
        raise RuntimeError("No episodes found.")

    log("INFO", f"Found {len(episodes)} episodes")

    if download_mode == "Single Episode":
        target = single_episode.strip()
        selected = [ep for ep in episodes if ep["id"] == target]
    elif download_mode == "Episode Range":
        if parse_episode_id(start_episode) > parse_episode_id(end_episode):
            raise ValueError(f"Invalid episode range: {start_episode} > {end_episode}")
        selected = [ep for ep in episodes if in_episode_range(ep["id"], start_episode, end_episode)]
    else:
        selected = episodes

    if not selected:
        raise RuntimeError("No episodes match your selection.")

    log("INFO", f"Will download {len(selected)} episode(s)")

    download_dir = os.path.join("downloads", anime_title)
    os.makedirs(download_dir, exist_ok=True)
    log("INFO", f"Download directory: {download_dir}")

    downloaded_files: List[str] = []
    failed_episodes: List[str] = []

    for idx, ep in enumerate(selected, 1):
        ep_id = ep["id"]
        print("\n" + "-" * 50)
        log("INFO", f"[{idx}/{len(selected)}] Episode {ep_id}")

        servers = get_video_servers(ep["token"])
        if not servers:
            log("ERROR", "No servers available for this episode.")
            failed_episodes.append(ep_id)
            continue

        server = choose_server(servers, prefer_type, prefer_server)
        if not server:
            log("ERROR", "Could not choose any server.")
            failed_episodes.append(ep_id)
            continue

        log("INFO", f"Using server: {server['server_name']} (type={server['type']})")

        # Get video data with subtitles
        video_data = get_video_data(server["server_id"])
        if not video_data:
            log("ERROR", "Could not resolve video data.")
            failed_episodes.append(ep_id)
            continue

        # FIXED: Generate proper filename
        filename = generate_episode_filename(anime_title, final_season, ep_id)
        filepath = os.path.join(download_dir, filename)

        log("INFO", f"Output filename: {filename}")

        if download_episode(video_data, filepath, ep_id):
            downloaded_files.append(filepath)
        else:
            log("ERROR", "All download attempts failed for this episode.")
            failed_episodes.append(ep_id)
        time.sleep(1)

    merged_video = None
    # Only merge if multiple episodes AND merge is enabled
    if merge_episodes and len(downloaded_files) > 1:
        # Create mapping with proper filenames
        file_by_ep = {}
        for ep in selected:
            filename = generate_episode_filename(anime_title, final_season, ep["id"])
            filepath = os.path.join(download_dir, filename)
            file_by_ep[ep["id"]] = filepath

        ordered_files = [file_by_ep[ep["id"]] for ep in selected if os.path.exists(file_by_ep[ep["id"]])]

        first_ep_id = selected[0]["id"]
        last_ep_id = selected[-1]["id"]
        merged_video = merge_videos(ordered_files, anime_title, final_season, first_ep_id, last_ep_id)

        if merged_video and not keep_individual_files:
            log("INFO", "Removing individual episode files after merge.")
            for fpath in ordered_files:
                try:
                    os.remove(fpath)
                except Exception:
                    pass

    print("\n" + "=" * 70)
    print("📊 DOWNLOAD SUMMARY")
    print("=" * 70)
    print(f"\n✅ Successfully downloaded: {len(downloaded_files)} episode(s)")
    if failed_episodes:
        print(f"❌ Failed episodes: {', '.join(failed_episodes)}")

    if merged_video:
        size_mb = os.path.getsize(merged_video) / (1024 * 1024)
        print(f"\n🔗 Merged file: {os.path.basename(merged_video)} ({size_mb:.2f} MB)")
        if not keep_individual_files:
            print("   Individual episode files were deleted.")
    elif downloaded_files:
        total = sum(os.path.getsize(f) for f in downloaded_files if os.path.exists(f)) / (1024 * 1024)
        print(f"\n💾 Total size of downloaded episodes: {total:.2f} MB")

    print(f"\n📁 Local files location: {download_dir}")

    # Show downloaded filenames
    if downloaded_files:
        print("\n📝 Downloaded files:")
        for fpath in downloaded_files:
            if os.path.exists(fpath):
                print(f"   • {os.path.basename(fpath)}")

    # FIXED: Upload logic for single episodes
    files_to_upload: List[str] = []

    # Determine which files to upload
    if len(downloaded_files) == 1:
        # Single episode - always upload it
        files_to_upload = downloaded_files
        log("INFO", "Single episode detected - will upload individual file")
    elif merge_episodes and len(downloaded_files) > 1:
        # Multiple episodes with merge enabled
        if upload_merged_only and merged_video:
            # Only upload merged file
            files_to_upload = [merged_video]
            log("INFO", "Upload merged only - will upload merged file")
        elif merged_video:
            # Upload merged + individuals if kept
            files_to_upload = [merged_video]
            if keep_individual_files:
                files_to_upload.extend([f for f in downloaded_files if os.path.exists(f)])
            log("INFO", f"Will upload merged file + {len(files_to_upload)-1} individual files")
        else:
            # Merge failed, upload individuals
            files_to_upload = [f for f in downloaded_files if os.path.exists(f)]
            log("INFO", "Merge failed - will upload individual files")
    else:
        # Multiple episodes, merge disabled - upload all
        files_to_upload = [f for f in downloaded_files if os.path.exists(f)]
        log("INFO", f"Merge disabled - will upload {len(files_to_upload)} individual files")

    gofile_links = []
    gdrive_paths = []

    if files_to_upload and upload_destination != "None (Keep Local)":
        print("\n" + "=" * 70)
        print("📤 UPLOADING FILES")
        print("=" * 70)
        print(f"\nFiles to upload: {len(files_to_upload)}")

        for path in files_to_upload:
            if not os.path.exists(path):
                log("WARN", f"File not found, skipping: {os.path.basename(path)}")
                continue

            print(f"\n📤 Processing: {os.path.basename(path)}")

            if upload_destination in ["GoFile.io Only", "Both"]:
                link = upload_to_gofile(path)
                if link:
                    gofile_links.append((os.path.basename(path), link))

            if upload_destination in ["Google Drive Only", "Both"]:
                dpath = upload_to_gdrive(path)
                if dpath:
                    gdrive_paths.append(os.path.basename(dpath))

        print("\n" + "=" * 70)
        print("✅ UPLOAD COMPLETE")
        print("=" * 70)
        if gofile_links:
            print("\n🔗 GoFile Links:")
            for name, link in gofile_links:
                print(f"   • {name}: {link}")
        if gdrive_paths:
            print("\n📁 Google Drive files (MyDrive/AnimeKai_Downloads/):")
            for name in gdrive_paths:
                print(f"   • {name}")
    elif upload_destination == "None (Keep Local)":
        print("\n📁 Upload disabled - files kept locally only")

    print("\n" + "=" * 70)
    print("🎉 ALL DONE!")
    print("=" * 70)
    print(f"\n📺 Anime: {anime_title}")
    print(f"📊 Season: {final_season}")
    print(f"📥 Downloaded: {len(downloaded_files)} episode(s)")
    if failed_episodes:
        print(f"❌ Failed: {', '.join(failed_episodes)}")
    if merged_video:
        print(f"🔗 Merged file: {os.path.basename(merged_video)}")
    print(f"\n📁 Local files: {download_dir}")

    # Show subtitle info if Soft Sub or Dub was selected
    if prefer_type in ["Soft Sub", "Dub (with subs)"]:
        print(f"\n💬 Subtitles: Embedded in video file(s) when available")
        print("   (Enable subtitles in your video player's subtitle menu)")

    print("\n" + "=" * 70)

# Run main
try:
    main()
except Exception as e:
    print("\n" + "=" * 70)
    print("❌ ERROR")
    print("=" * 70)
    log("ERROR", f"Fatal error: {e}")
    import traceback
    traceback.print_exc()
    print("\n" + "=" * 70)

In [ ]:

# 🎬 Video Episode Merger & Uploader
# Download ZIP file with video episodes, extract, merge them in order, and upload

# @title 🔧 **Install Dependencies** { display-mode: "form" }
print("📦 Installing required packages...")
!pip install -q natsort requests
!apt-get -qq install -y ffmpeg > /dev/null 2>&1
print("✅ All dependencies installed!\n")

# @title ⚙️ **Configuration** { display-mode: "form" }

#@markdown ### 📥 Download Settings
#@markdown Enter the direct download link (DDL) for your ZIP file:
zip_url = "https://example.com/videos.zip" #@param {type:"string"}

#@markdown Custom User-Agent (leave default if unsure):
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" #@param {type:"string"}

#@markdown ---
#@markdown ### 🎬 Merge Settings
#@markdown Custom output name (leave empty to auto-detect from episodes):
custom_output_name = "" #@param {type:"string"}

#@markdown Video quality for merge:
merge_quality = "Copy Original (Fastest)" #@param ["Copy Original (Fastest)", "Re-encode High Quality", "Re-encode Compressed"]

#@markdown ---
#@markdown ### 📤 Upload Settings
upload_destination = "Both (GoFile + Google Drive)" #@param ["GoFile.io Only", "Google Drive Only", "Both (GoFile + Google Drive)", "None (Keep Local Only)"]

#@markdown Create ZIP of merged video for upload?
create_upload_zip = False #@param {type:"boolean"}

print("✅ Configuration set!")

# @title 📥 **Download ZIP File** { display-mode: "form" }
import requests
import os
import re
from urllib.parse import unquote, urlparse
from pathlib import Path

print("🔽 Starting download...")
print(f"🔗 URL: {zip_url[:60]}..." if len(zip_url) > 60 else f"🔗 URL: {zip_url}")

headers = {'User-Agent': user_agent}

try:
    response = requests.get(zip_url, headers=headers, stream=True, allow_redirects=True)
    response.raise_for_status()

    # Smart filename detection
    zip_filename = None

    # Method 1: Content-Disposition header
    if 'Content-Disposition' in response.headers:
        cd = response.headers['Content-Disposition']
        filenames = re.findall(r'filename\*?=["\']?(?:UTF-8\'\')?([^"\';]+)["\']?', cd)
        if filenames:
            zip_filename = unquote(filenames[0])
            print(f"📋 Filename from header: {zip_filename}")

    # Method 2: Final URL after redirects
    if not zip_filename:
        final_url = response.url
        url_path = urlparse(final_url).path
        zip_filename = os.path.basename(url_path)
        zip_filename = unquote(zip_filename)
        print(f"📋 Filename from URL: {zip_filename}")

    # Method 3: Extract meaningful name from URL
    if not zip_filename or zip_filename in ['', 'download', 'file']:
        # Try to extract from full URL path
        url_parts = [p for p in urlparse(zip_url).path.split('/') if p and p != 'download']
        if url_parts:
            zip_filename = url_parts[-1]
            zip_filename = unquote(zip_filename)

    # Ensure .zip extension
    if not zip_filename.lower().endswith('.zip'):
        if '.' not in zip_filename:
            zip_filename += '.zip'
        else:
            zip_filename = os.path.splitext(zip_filename)[0] + '.zip'

    # Clean filename (remove invalid characters)
    zip_filename = re.sub(r'[<>:"|?*\\]', '_', zip_filename)
    zip_filename = re.sub(r'[\x00-\x1f]', '', zip_filename)  # Remove control characters

    print(f"💾 Saving as: {zip_filename}")

    total_size = int(response.headers.get('content-length', 0))
    downloaded = 0

    with open(zip_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                downloaded += len(chunk)
                if total_size:
                    percent = (downloaded / total_size) * 100
                    mb_downloaded = downloaded / (1024*1024)
                    mb_total = total_size / (1024*1024)
                    print(f"\r⏳ Progress: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')

    print(f"\n✅ Downloaded: {zip_filename} ({downloaded / (1024*1024):.2f} MB)")

    # Extract base name for later use
    ZIP_BASE_NAME = os.path.splitext(zip_filename)[0]
    ZIP_BASE_NAME = re.sub(r'[_\-\s]+', ' ', ZIP_BASE_NAME).strip()

    print(f"📦 Base name extracted: '{ZIP_BASE_NAME}'")

except Exception as e:
    print(f"\n❌ Download failed: {str(e)}")
    import traceback
    print(traceback.format_exc())
    raise

# @title 📦 **Extract ZIP File** { display-mode: "form" }
import zipfile

extract_folder = "extracted_videos"
os.makedirs(extract_folder, exist_ok=True)

print(f"\n📂 Extracting to: {extract_folder}/")
print("⏳ Please wait...")

try:
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        total_files = len(file_list)

        print(f"📋 Found {total_files} file(s) in ZIP\n")

        # Extract with progress
        for idx, file in enumerate(file_list, 1):
            zip_ref.extract(file, extract_folder)
            if idx % 5 == 0 or idx == total_files:
                print(f"\r⏳ Extracting: {idx}/{total_files} files...", end='')

        print(f"\n\n📄 Extracted files:")
        video_count = 0
        for file in file_list:
            file_lower = file.lower()
            is_video = any(file_lower.endswith(ext) for ext in ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.m4v'])
            icon = "🎬" if is_video else "📄"
            print(f"  {icon} {file}")
            if is_video:
                video_count += 1

        print(f"\n✅ Extraction complete! Found {video_count} video file(s)")

except Exception as e:
    print(f"\n❌ Extraction failed: {str(e)}")
    raise

# @title 🔍 **Detect and Sort Episodes** { display-mode: "form" }
import re
from natsort import natsorted

def extract_episode_info(filename):
    """Enhanced episode detection with better pattern matching"""
    name = os.path.basename(filename)

    # Combined season and episode patterns (S01E01, S1E1, etc.)
    combined_patterns = [
        r'[Ss](\d+)[Ee](\d+)',  # S01E01, S1E1
        r'[Ss]eason[\s._-]*(\d+)[\s._-]*[Ee]pisode[\s._-]*(\d+)',  # Season 1 Episode 1
        r'[Ss]eason[\s._-]*(\d+)[\s._-]*[Ee][Pp][\s._-]*(\d+)',  # Season 1 Ep 1
        r'(\d+)[xX](\d+)',  # 1x01
    ]

    # Try combined patterns first
    for pattern in combined_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            return int(match.group(1)), int(match.group(2))

    # Separate season patterns
    season_patterns = [
        r'[Ss]eason[\s._-]*(\d+)',
        r'[Ss](\d+)(?![Ee])',  # S1 but not followed by E
        r'Season[\s._-]*(\d+)',
    ]

    # Episode patterns
    episode_patterns = [
        r'[Ee]pisode[\s._-]*(\d+)',
        r'[Ee][Pp][\s._-]*(\d+)',
        r'[Ee](\d+)',
        r'Episode[\s._-]*(\d+)',
        r'[\s._-](\d{1,3})[\s._-]',  # Number surrounded by separators
        r'^(\d{1,3})[\s._-]',  # Number at start
        r'[\s._-](\d{1,3})\.',  # Number before extension
    ]

    season = None
    episode = None

    # Find season
    for pattern in season_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            season = int(match.group(1))
            break

    # Find episode
    for pattern in episode_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            ep_num = int(match.group(1))
            # Reasonable episode number (1-999)
            if 1 <= ep_num <= 999:
                episode = ep_num
                break

    return season, episode

# Find all video files
video_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.m4v', '.ts', '.m2ts']
video_files = []

for root, dirs, files in os.walk(extract_folder):
    for file in files:
        if any(file.lower().endswith(ext) for ext in video_extensions):
            full_path = os.path.join(root, file)
            video_files.append(full_path)

if not video_files:
    print("❌ No video files found in the ZIP!")
    raise Exception("No video files detected")

print(f"🎬 Found {len(video_files)} video file(s)\n")

# Extract info and sort
video_info = []
for vf in video_files:
    season, episode = extract_episode_info(vf)
    video_info.append({
        'path': vf,
        'name': os.path.basename(vf),
        'season': season if season else 0,
        'episode': episode if episode else 0
    })

# Sort by season, then episode, then natural name
video_info.sort(key=lambda x: (x['season'], x['episode'], x['name']))

print("📋 **Detected Episode Order:**")
print("=" * 70)
for idx, info in enumerate(video_info, 1):
    s_info = f"S{info['season']:02d}" if info['season'] else "S??"
    e_info = f"E{info['episode']:02d}" if info['episode'] else "E??"
    size_mb = os.path.getsize(info['path']) / (1024*1024)
    print(f"{idx:2d}. [{s_info}{e_info}] {info['name'][:45]:<45} ({size_mb:.1f} MB)")
print("=" * 70)

# @title 🎞️ **Merge Videos** { display-mode: "form" }
import subprocess

print("\n🎬 Preparing to merge videos...")

# Create file list for ffmpeg
list_file = "filelist.txt"
with open(list_file, 'w', encoding='utf-8') as f:
    for info in video_info:
        # Escape single quotes for ffmpeg
        safe_path = info['path'].replace("'", "'\\''")
        f.write(f"file '{safe_path}'\n")

print(f"✅ Created merge list with {len(video_info)} video(s)")

# Determine output filename
if custom_output_name:
    output_name = custom_output_name
    if not output_name.lower().endswith('.mp4'):
        output_name += '.mp4'
else:
    # Auto-generate name
    seasons = [v['season'] for v in video_info if v['season'] > 0]
    episodes = [v['episode'] for v in video_info if v['episode'] > 0]

    base_name = ZIP_BASE_NAME

    if seasons and episodes:
        min_season = min(seasons)
        max_season = max(seasons)
        min_episode = min(episodes)
        max_episode = max(episodes)

        if min_season == max_season:
            output_name = f"{base_name} Season {min_season:02d} Episodes {min_episode:02d}-{max_episode:02d}.mp4"
        else:
            output_name = f"{base_name} S{min_season:02d}-S{max_season:02d} Ep{min_episode:02d}-{max_episode:02d}.mp4"
    else:
        output_name = f"{base_name} Merged Complete.mp4"

# Clean output name
output_name = re.sub(r'[<>:"|?*\\]', '_', output_name)
output_name = re.sub(r'\s+', ' ', output_name).strip()

print(f"\n📁 Output filename: {output_name}")

# Build ffmpeg command based on quality setting
if merge_quality == "Copy Original (Fastest)":
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c', 'copy', output_name, '-y'
    ]
    print("⚡ Mode: Fast merge (copy streams, no re-encoding)")
elif merge_quality == "Re-encode High Quality":
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c:v', 'libx264', '-crf', '18', '-preset', 'slow',
        '-c:a', 'aac', '-b:a', '192k',
        output_name, '-y'
    ]
    print("🎨 Mode: High quality re-encode (slower, best quality)")
else:  # Compressed
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c:v', 'libx264', '-crf', '23', '-preset', 'medium',
        '-c:a', 'aac', '-b:a', '128k',
        output_name, '-y'
    ]
    print("📦 Mode: Compressed re-encode (smaller file size)")

print("\n⏳ Merging videos... This may take a while.\n")

try:
    # Run ffmpeg
    process = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True)

    # Parse ffmpeg output for progress
    duration_pattern = re.compile(r'Duration: (\d{2}):(\d{2}):(\d{2})')
    time_pattern = re.compile(r'time=(\d{2}):(\d{2}):(\d{2})')

    total_duration = None

    for line in process.stderr:
        # Get total duration
        if total_duration is None:
            dur_match = duration_pattern.search(line)
            if dur_match:
                h, m, s = map(int, dur_match.groups())
                total_duration = h * 3600 + m * 60 + s

        # Get current time
        time_match = time_pattern.search(line)
        if time_match and total_duration:
            h, m, s = map(int, time_match.groups())
            current_time = h * 3600 + m * 60 + s
            percent = (current_time / total_duration) * 100
            print(f"\r🎬 Progress: {percent:.1f}% ({current_time//60}:{current_time%60:02d} / {total_duration//60}:{total_duration%60:02d})", end='')

    process.wait()

    if process.returncode == 0:
        file_size = os.path.getsize(output_name) / (1024*1024)
        print(f"\n\n✅ **Merge Complete!**")
        print("=" * 70)
        print(f"📁 Output: {output_name}")
        print(f"💾 Size: {file_size:.2f} MB")
        print(f"🎬 Episodes: {len(video_info)}")
        print("=" * 70)

        MERGED_VIDEO = output_name
    else:
        print(f"\n❌ Merge failed with exit code {process.returncode}")
        raise Exception("FFmpeg merge failed")

except Exception as e:
    print(f"\n❌ Error during merge: {str(e)}")
    raise
finally:
    # Cleanup
    if os.path.exists(list_file):
        os.remove(list_file)

# @title 📦 **Create ZIP of Merged Video (Optional)** { display-mode: "form" }

if create_upload_zip:
    print("\n📦 Creating ZIP file of merged video...")

    zip_output = output_name.replace('.mp4', '.zip')

    import zipfile
    with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED, compresslevel=0) as zipf:
        print(f"⏳ Adding {output_name} to ZIP...")
        zipf.write(output_name, os.path.basename(output_name))

    zip_size = os.path.getsize(zip_output) / (1024*1024)
    print(f"✅ ZIP created: {zip_output} ({zip_size:.2f} MB)")

    UPLOAD_FILE = zip_output
else:
    UPLOAD_FILE = MERGED_VIDEO
    print("\n📄 Will upload video file directly (no ZIP)")

# @title 📤 **Upload Files** { display-mode: "form" }

def upload_to_gofile(filepath):
    """Upload file to GoFile.io"""
    try:
        print("\n🌐 GoFile.io Upload")
        print("-" * 50)

        # Get best server
        server_response = requests.get('https://api.gofile.io/servers', timeout=30)
        server_response.raise_for_status()
        server_data = server_response.json()

        if server_data['status'] != 'ok':
            print("❌ Failed to get GoFile server")
            return None

        server = server_data['data']['servers'][0]['name']
        print(f"📡 Server: {server}")

        # Updated endpoint
        upload_url = f'https://{server}.gofile.io/contents/uploadfile'

        file_size_mb = os.path.getsize(filepath) / (1024*1024)
        print(f"📦 File: {os.path.basename(filepath)} ({file_size_mb:.2f} MB)")
        print("⏳ Uploading... (this may take several minutes for large files)")

        with open(filepath, 'rb') as f:
            files_data = {'file': (os.path.basename(filepath), f, 'application/octet-stream')}
            response = requests.post(upload_url, files=files_data, timeout=7200)

        response.raise_for_status()

        result = response.json()
        if result['status'] == 'ok':
            download_page = result['data']['downloadPage']
            print("✅ Upload successful!")
            print(f"🔗 Link: {download_page}")
            return download_page
        else:
            print(f"❌ Upload failed: {result.get('message', 'Unknown error')}")
            return None

    except requests.exceptions.Timeout:
        print("❌ Upload timed out - file may be too large for GoFile")
        return None
    except requests.exceptions.JSONDecodeError:
        print("❌ Invalid response from GoFile - service may be down")
        return None
    except Exception as e:
        print(f"❌ GoFile error: {str(e)}")
        return None

def upload_to_gdrive(filepath):
    """Upload file to Google Drive"""
    try:
        print("\n☁️ Google Drive Upload")
        print("-" * 50)

        from google.colab import drive

        # Check if already mounted
        if not os.path.exists('/content/drive/MyDrive'):
            drive.mount('/content/drive', force_remount=False)
            print("✅ Google Drive mounted!")
        else:
            print("✅ Google Drive already mounted!")

        destination = '/content/drive/MyDrive/Merged_Videos/'
        os.makedirs(destination, exist_ok=True)

        dest_path = os.path.join(destination, os.path.basename(filepath))

        # Check if file exists
        if not os.path.exists(filepath):
            print(f"❌ Source file not found: {filepath}")
            return None

        file_size_mb = os.path.getsize(filepath) / (1024*1024)
        print(f"📦 File: {os.path.basename(filepath)} ({file_size_mb:.2f} MB)")
        print(f"⏳ Copying to Google Drive...")

        import shutil
        shutil.copy2(filepath, dest_path)

        print("✅ Upload successful!")
        print(f"📁 Location: MyDrive/Merged_Videos/{os.path.basename(filepath)}")
        return dest_path

    except Exception as e:
        print(f"❌ Google Drive error: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

# Execute uploads based on user selection
print("\n" + "=" * 70)
print("📤 UPLOAD PROCESS")
print("=" * 70)

# Track upload results
gofile_link = None
gdrive_path = None

if upload_destination == "GoFile.io Only":
    gofile_link = upload_to_gofile(UPLOAD_FILE)

elif upload_destination == "Google Drive Only":
    gdrive_path = upload_to_gdrive(UPLOAD_FILE)

elif upload_destination == "Both (GoFile + Google Drive)":
    gofile_link = upload_to_gofile(UPLOAD_FILE)
    gdrive_path = upload_to_gdrive(UPLOAD_FILE)

else:  # None
    print("\n📁 Upload skipped - file saved locally")
    print(f"📄 Location: /content/{UPLOAD_FILE}")

# Summary
print("\n" + "=" * 70)
print("🎉 **ALL DONE!**")
print("=" * 70)
print(f"\n📊 Summary:")
print(f"  • Videos merged: {len(video_info)}")
print(f"  • Output file: {output_name}")
print(f"  • File size: {os.path.getsize(MERGED_VIDEO) / (1024*1024):.2f} MB")

if gofile_link:
    print(f"\n🔗 GoFile.io Link:")
    print(f"   {gofile_link}")

if gdrive_path:
    print(f"\n📁 Google Drive:")
    print(f"   {gdrive_path}")

if not gofile_link and not gdrive_path and upload_destination != "None (Keep Local Only)":
    print(f"\n⚠️ Note: Some uploads may have failed. Check error messages above.")

print("\n✨ Process complete!")